In [16]:
import math
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

**This is the start of Tutorial 2**

In [17]:
## Define Variables

B = 150 # channel width (m)
H = 8 # flow depth (m)
S = 0.0002 # channel slope (m/m)
Cf = 60 # Chezy friction coefficient
D50 = 0.001 # median grain size (m)
shareOfIjssel = 0.16
Qlow = 3200*shareOfIjssel # minimum discharge (m^3/s)
Qhigh = 20000*shareOfIjssel # maximum discharge (m^3/s)
R = 1.65 # submerged specific gravity of sediment
g = 9.81 # gravitational acceleration (m/s^2)
rho = 1000 # water density (kg/m^3)

# main channel trapezoid slopes
rho_L = 0.1
rho_R = 0.1


In [ ]:
#functions

def chezy_rec(H,B,S,Cf):
    A = H*B
    P = B+2*H
    R = A/P
    u = Cf*np.sqrt(R*S)
    return u

def chezy_trap(H,B,S,Cf,rho_L,rho_R):
    hor_L = H/rho_L
    hor_R = H/rho_R
    A = B*H + H*hor_L/2 + H*hor_R/2
    P = B + np.sqrt(H**2+hor_L**2) + np.sqrt(H**2+hor_R**2)
    R = A/P
    u = Cf*np.sqrt(R*S)
    return u

def getDischarge_rec(H,B,S,Cf):
    A = H*B
    u = chezy_rec(H,B,S,Cf)
    Q = u*A
    return Q

def getDepth_rec(Q,B,S,Cf,tol=0.01):
    h = 1
    q = getDischarge_rec(h,B,S,Cf)
    while abs(Q-q) > tol:
        h = h*np.sqrt(Q/q)
        q = getDischarge_rec(h,B,S,Cf)
    return h

def getDischarge_trap(H,B,S,Cf,rho_L,rho_R):
    A = B*H + H**2/2/rho_L + H**2/2/rho_R
    u = chezy_trap(H,B,S,Cf,rho_L,rho_R)
    Q = u*A
    return Q

def getDepth_trap(Q,B,S,Cf,rho_L,rho_R,tol=0.01):
    h = 1
    q = getDischarge_trap(h,B,S,Cf,rho_L,rho_R)
    while abs(Q-q) > tol:
        h = h*np.sqrt(Q/q)
        q = getDischarge_trap(h,B,S,Cf,rho_L,rho_R)
    return h

# energy loss due to friction
def friction_loss(H,B,Cf,rho_L,rho_R,L,Q):
    hor_L = H/rho_L
    hor_R = H/rho_R
    A = B*H + H*hor_L/2 + H*hor_R/2
    P = B + np.sqrt(H**2+hor_L**2) + np.sqrt(H**2+hor_R**2)
    R = A/P

    i = Q**2/A**2/(Cf**2*R)
    delta_E = i*L
    return delta_E


In [35]:

Q101 = getDischarge_trap(H,B,S,Cf,rho_L,rho_R)
print("No side channel, bankful (no safety) discharge capacity:",round(Q101))

safetyHeight = 1 #m
Q102 = getDischarge_trap(H-safetyHeight,B,S,Cf,rho_L,rho_R)
print("No side channel, discharge capacity with 1m safety:",round(Q102))

print("High water total design discharge:",Qhigh)

print("Minimum discharge in side channel:",round(Qhigh-Q102))

B_side = 40
rho_side = 0.1
Cf_side = Cf
S_side = S
Q_side = Qhigh - Q102
print("Required side channel water depth for design discharge:",round(getDepth_trap(Q_side,B_side,S_side,Cf_side,rho_side,rho_side),2))

No side channel, bankful (no safety) discharge capacity: 3799
No side channel, discharge capacity with 1m safety: 3008
High water total design discharge: 3200.0
Minimum discharge in side channel: 192
Required side channel water depth for design discharge: 2.55


In [ ]:
# Low discharge check

rho = rho_L
print("Main channel water depth in case of low flow, no side channel: ",round(getDepth_trap(Qlow,B,S,Cf,rho,rho),2),"m")



Main channel water depth in case of low flow, no side channel:  2.39 m


In [46]:
Lm = 5000 # length of main channel reach (m)
Ls = 10000 # length of side channel reach (m) 
S_side = S*Lm/Ls


# Let's say the bottom of the channels is continuous, so the water depth in the two channels will be equal. No weir yet

# change in water surface elevation in main channel: 
Qtotal = Qhigh

# guess depth
h0 = 2
Qm = getDischarge_trap(h0,B,S,Cf,rho_L,rho_R)
Qs = getDischarge_trap(h0,B_side,S_side,Cf_side,rho_side,rho_side)
while abs(Qtotal - (Qm+Qs)) > 0.01:
    h0 = h0 * np.sqrt(Qtotal/(Qm+Qs))
    Qm = getDischarge_trap(h0,B,S,Cf,rho_L,rho_R)
    Qs = getDischarge_trap(h0,B_side,S_side,Cf_side,rho_side,rho_side)
print("Qtotal = ",Qtotal,"was split into Qmain = ",round(Qm,2),"and Qside = ",round(Qs,2),"at water depth h = ",round(h0,2),"m")
# check energy loss due to friction in both channels
delta_Em = friction_loss(h0,B,Cf,rho_L,rho_R,Lm,Qm)
delta_Es = friction_loss(h0,B_side,Cf_side,rho_side,rho_side,Ls,Qs)

print("Energy loss in main channel: ",delta_Em,"m", ", in side channel: ",delta_Es,"m")
# for low flow

Qtotal = Qlow
# guess depth
h0 = 2
Qm = getDischarge_trap(h0,B,S,Cf,rho_L,rho_R)
Qs = getDischarge_trap(h0,B_side,S_side,Cf_side,rho_side,rho_side)
while abs(Qtotal - (Qm+Qs)) > 0.01:
    h0 = h0 * np.sqrt(Qtotal/(Qm+Qs))
    Qm = getDischarge_trap(h0,B,S,Cf,rho_L,rho_R)
    Qs = getDischarge_trap(h0,B_side,S_side,Cf_side,rho_side,rho_side)
print("Qtotal = ",Qtotal,"was split into Qmain = ",round(Qm,2),"and Qside = ",round(Qs,2),"at water depth h = ",round(h0,2),"m")

# check energy loss due to friction in both channels
delta_Em = friction_loss(h0,B,Cf,rho_L,rho_R,Lm,Qm)
delta_Es = friction_loss(h0,B_side,Cf_side,rho_side,rho_side,Ls,Qs)

print("Energy loss in main channel: ",delta_Em,"m", ", in side channel: ",delta_Es,"m")

Qtotal =  3200.0 was split into Qmain =  2451.97 and Qside =  748.03 at water depth h =  6.22 m
Energy loss in main channel:  1.0 m , in side channel:  0.9999999999999998 m
Qtotal =  512.0 was split into Qmain =  416.14 and Qside =  95.85 at water depth h =  2.1 m
Energy loss in main channel:  0.9999999999999998 m , in side channel:  1.0000000000000002 m


In [68]:
weirHeight = 2 # weir height (m)
weirWidth = B_side # weir width (m)
mu = 0.74 # weir discharge coefficient
# it will probably be alulról befolyásolt in high flow, but simply átbukó in low flow. 

print("-------------------------- high discharge ----------------------------------")
Qtotal = Qhigh
# guess depth
h0 = 3
Qm = getDischarge_trap(h0,B,S,Cf,rho_L,rho_R)
h_weir = h0 - weirHeight
Q_weir = (2/3)*mu*math.sqrt(2*g)*weirWidth*h_weir**(3/2)
while abs(Qtotal - (Qm+Q_weir)) > 0.01:
    h0 = h0 * np.sqrt(Qtotal/(Qm+Q_weir))
    Qm = getDischarge_trap(h0,B,S,Cf,rho_L,rho_R)
    h_weir = h0 - weirHeight
    Q_weir = (2/3)*mu*math.sqrt(2*g)*weirWidth*h_weir**(3/2)
print(" Qtotal =",Qtotal,", Qmain = ",round(Qm,2),"and Qweir = ",round(Q_weir,2),"at water depth h = ",round(h0,2),"m")

delta_Em = friction_loss(h0,B,Cf,rho_L,rho_R,Lm,Qm)
delta_Es = friction_loss(h0,B_side,Cf_side,rho_side,rho_side,Ls,Q_weir)
delta_Ew = Q_weir**2/(g*(mu*h_weir*B_side)**2)

print("Depth in side channel:",getDepth_trap(Q_weir,B_side,S_side,Cf_side,rho_side,rho_side))
print("The flow over the weir would definitely be influenced from below in high flow. ")
print("Energy loss in main channel: ",delta_Em,"m", ", over weir and in side channel:",delta_Es+delta_Ew,"m")

print("-------------------------- low discharge ----------------------------------")


# get an initial state
Qtotal = Qlow
# guess depth
h0 = 3
Qm = getDischarge_trap(h0,B,S,Cf,rho_L,rho_R)
h_weir = h0 - weirHeight
Q_weir = (2/3)*mu*math.sqrt(2*g)*weirWidth*h_weir**(3/2)
while abs(Qtotal - (Qm+Q_weir)) > 0.01:
    h0 = h0 * np.sqrt(Qtotal/(Qm+Q_weir))
    Qm = getDischarge_trap(h0,B,S,Cf,rho_L,rho_R)
    h_weir = h0 - weirHeight
    Q_weir = (2/3)*mu*math.sqrt(2*g)*weirWidth*h_weir**(3/2)
print(" Qtotal =",Qtotal,", Qmain = ",round(Qm,2),"and Qweir = ",round(Q_weir,2),"at water depth h = ",round(h0,2),"m")

delta_Em = friction_loss(h0,B,Cf,rho_L,rho_R,Lm,Qm)
delta_Es = friction_loss(h0,B_side,Cf_side,rho_side,rho_side,Ls,Q_weir)
delta_Ew = Q_weir**2/(g*(mu*h_weir*B_side)**2)

print("Depth in side channel:",getDepth_trap(Q_weir,B_side,S_side,Cf_side,rho_side,rho_side))
print("Weir equation holds for low flow, no influence from below.")


-------------------------- high discharge ----------------------------------
 Qtotal = 3200.0 , Qmain =  2445.47 and Qweir =  754.53 at water depth h =  6.21 m
Depth in side channel: 6.244213476693916
The flow over the weir would definitely be influenced from below in high flow. 
Energy loss in main channel:  1.0000000000000002 m , over weir and in side channel: 4.764409975871846 m
-------------------------- low discharge ----------------------------------
 Qtotal = 512.0 , Qmain =  494.56 and Qweir =  17.44 at water depth h =  2.34 m
Depth in side channel: 0.7568294520450181
Weir equation holds for low flow, no influence from below.


In [ ]:
# new way to calculate flow over weir

